In [2]:
import json
import pandas as pd
from copy import deepcopy
from pprint import pprint
from utils import extract_integer
import os

# Convert all the Json File to CSV

In [4]:
json_file_path = "data/vehiclePosition08.json"

# Reading JSON file
with open(json_file_path) as json_file:
    json_dict = json.load(json_file)

In [5]:
## Information about the lineIds (of buses, train, tram)
# LineId: The unique number of the line route
# directionId: the direction in which the specific vehicle is traveling
# pointId: The last stop travered, stop which is left behind
# distanceFromPoint: Distance from the left behind stop

In [6]:
single_line_info: dict = json_dict["data"][0]["Responses"][0]
single_line_info

{'lines': [{'lineId': '1',
   'vehiclePositions': [{'directionId': '8731',
     'distanceFromPoint': 0,
     'pointId': '8162'},
    {'directionId': '8731', 'distanceFromPoint': 0, 'pointId': '8131'},
    {'directionId': '8162', 'distanceFromPoint': 0, 'pointId': '8092'},
    {'directionId': '8731', 'distanceFromPoint': 0, 'pointId': '8011'},
    {'directionId': '8161', 'distanceFromPoint': 1, 'pointId': '8122'},
    {'directionId': '8161', 'distanceFromPoint': 0, 'pointId': '8742'},
    {'directionId': '8731', 'distanceFromPoint': 0, 'pointId': '8161'},
    {'directionId': '8731', 'distanceFromPoint': 1, 'pointId': '8101'},
    {'directionId': '8731', 'distanceFromPoint': 0, 'pointId': '8051'},
    {'directionId': '8731', 'distanceFromPoint': 0, 'pointId': '8291'},
    {'directionId': '8162', 'distanceFromPoint': 0, 'pointId': '8272'},
    {'directionId': '8161', 'distanceFromPoint': 0, 'pointId': '8052'}]},
  {'lineId': '2',
   'vehiclePositions': [{'directionId': '8763',
     'dista

In [19]:
raw_json_data_dir = "data/"
for json_file in os.listdir(raw_json_data_dir):
    if ".json" in json_file:
        print(json_file)

vehiclePosition05.json
vehiclePosition01.json
vehiclePosition09.json
vehiclePosition03.json
vehiclePosition12.json
vehiclePosition08.json
vehiclePosition10.json
vehiclePosition04.json
vehiclePosition13.json
vehiclePosition07.json
vehiclePosition11.json
vehiclePosition02.json
vehiclePosition06.json


In [21]:
info_for_line_ids = []

raw_json_data_dir = "data"
for json_file_path in os.listdir(raw_json_data_dir):
    if ".json" in json_file_path:
        print(f'Running on JSON file: {json_file_path}')

        # Reading JSON file
        with open(f'{raw_json_data_dir}/{json_file_path}') as json_file:
            json_dict = json.load(json_file)
    
        for data in json_dict["data"]:
            timestamp = data["time"]
            # Rest of the stuff
            for upper_line_group in data["Responses"]:
                # print(upper_line_group)
                if upper_line_group is None:
                    continue
                for line_group in upper_line_group["lines"]:
        #             print(line_group)
                    line_id_info = [{"timestamp": timestamp, "lineId": line_group["lineId"], **elem}
                                    for elem in line_group["vehiclePositions"]]

                    info_for_line_ids.extend(deepcopy(line_id_info))
        lines_df = pd.DataFrame(info_for_line_ids)
        print(f"Done CSV len: {len(lines_df)}")
        # lines_df.to_csv(f"vehiclePosition{i}.csv", index=False)
        lines_df.head()

Running on JSON file: vehiclePosition05.json
Done CSV len: 672800
Running on JSON file: vehiclePosition01.json
Done CSV len: 2038411
Running on JSON file: vehiclePosition09.json
Done CSV len: 4701492
Running on JSON file: vehiclePosition03.json
Done CSV len: 5114311
Running on JSON file: vehiclePosition12.json
Done CSV len: 6787207
Running on JSON file: vehiclePosition08.json
Done CSV len: 6933811
Running on JSON file: vehiclePosition10.json
Done CSV len: 10482733
Running on JSON file: vehiclePosition04.json
Done CSV len: 11411801
Running on JSON file: vehiclePosition13.json
Done CSV len: 11695815
Running on JSON file: vehiclePosition07.json
Done CSV len: 12811720
Running on JSON file: vehiclePosition11.json
Done CSV len: 13922929
Running on JSON file: vehiclePosition02.json
Done CSV len: 16826988
Running on JSON file: vehiclePosition06.json
Done CSV len: 19421883


In [3]:
proc_lines = pd.read_csv("processed_data/vehiclePosition08.csv")
idx_lineId_95 = proc_lines["lineId"] == 95
line_95 = deepcopy( proc_lines.loc[idx_lineId_95] )

# 3. Given a vehicle start time, do arrival time forecasting at a given stop in the route of this vehicle. You should be able to test the accuracy of your

### Input to this section is a file that has vehicles trajectory(ies)Mm

In [3]:
traj_df = pd.read_csv("bus_95_trajectory.csv")
traj_df.head()
# Filter trajectory of 1 bus
idxs = traj_df["bus_id"] == 71
a_vehicle_traj = deepcopy( traj_df.loc[idxs, :] )
a_vehicle_traj.head()

,timestamp,lineId,directionId,pointId,istop_id,distanceFromPoint,et,bus_id
811,1631611367813,95,4318,6354,5,790,2021-09-14 09:22:47.813,71
818,1631611399456,95,4318,6433,6,9,2021-09-14 09:23:19.456,71
824,1631611429562,95,4318,1729,7,19,2021-09-14 09:23:49.562,71
830,1631611460999,95,4318,1729,7,186,2021-09-14 09:24:20.999,71
836,1631611491106,95,4318,1233,8,0,2021-09-14 09:24:51.106,71


In [4]:
a_vehicle_traj.loc[:, "et"] = a_vehicle_traj.et.apply(lambda x: pd.to_datetime(x))
a_vehicle_traj.et.diff().describe()

count                           49
mean     0 days 00:00:31.654938775
std      0 days 00:00:00.975767249
min         0 days 00:00:30.105000
25%         0 days 00:00:30.925000
50%         0 days 00:00:31.643000
75%         0 days 00:00:32.469000
max         0 days 00:00:33.485000
Name: et, dtype: object